In [10]:
import os
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime

def load_data_at_altitude(altitude, data_folder, latitudes, longitudes):
    # List all files in the folder
    file_list = os.listdir(data_folder)

    # Filter the files to keep only those with the correct naming convention
    filtered_files = [file for file in file_list if file.endswith('.npz')  and '2009-01' in file ] #and '2018-02' in file]

    # Sort the files by date
    filtered_files.sort()
    print(f'{data_folder}/{filtered_files[0]}')
    # Load all files into a single xarray Dataset

    data_arrays = []
    for file in filtered_files:
        print(file)
        data_array = xr.DataArray(
            np.load(f'{data_folder}/{file}')['arr_0'],
            coords={
                'latitude': (('latitude'), latitudes),
                'longitude': (('longitude'), longitudes),
                'time': datetime.strptime(file[:10], '%Y-%m-%d'),
                'altitude': altitude
            },
            dims=('latitude', 'longitude'),
            attrs={'altitude': altitude}
        )
        data_arrays.append(data_array)

    combined_data = xr.concat(data_arrays, dim='time')
    return combined_data

# Set the folder paths
data_folder_24km = "processed_airs_sftp/40_km_grid/24km/mfx"
data_folder_30km = "processed_airs_sftp/40_km_grid/30km/mfx/"
data_folder_36km = "processed_airs_sftp/40_km_grid/36km/mfx"
data_folder_42km = "processed_airs_sftp/40_km_grid/42km/mfx"
data_folder_climatology_airs = "climatology/airs/30km/mfx"

# lat = "coordinates_40km_grid/Airs_nh_lat_40_grid.csv"
# lon = "coordinates_40km_grid/Airs_nh_lon_40_grid.csv"

lat = 'lat.csv'
lon = 'lon.csv'

# Read the latitude and longitude data from the CSV file
latitudes = np.loadtxt(lat, delimiter=',', unpack=True)
longitudes = np.loadtxt(lon, delimiter=',', unpack=True)

# Load data at 30km and 40km altitudes
# data_24km = load_data_at_altitude(24, data_folder_24km, latitudes, longitudes)
data_30km = load_data_at_altitude(30, data_folder_30km, latitudes, longitudes)
# data_36km = load_data_at_altitude(36, data_folder_36km, latitudes, longitudes)
# data_climatology = load_data_at_altitude(30, data_folder_climatology_airs, latitudes, longitudes)

print('concat data')
# Combine data at both altitudes
combined_data = xr.concat([data_30km], dim='altitude')

# Create an xarray Dataset with a data variable named 'mfx'
combined_dataset = xr.Dataset({'mfx': combined_data})

# Save the combined dataset to a new .nc file
# combined_dataset.to_netcdf('combined_data_with_coordinates.nc')
print('save to net_cdf')

combined_dataset.to_netcdf('airs_30km_climatology.nc')

print('All data has been combined into a single .nc file with coordinates.')


processed_airs_sftp/40_km_grid/30km/mfx//2009-01-01_mfx.npz
2009-01-01_mfx.npz
2009-01-01_tp.npz
2009-01-02_mfx.npz
2009-01-02_tp.npz
2009-01-03_mfx.npz
2009-01-03_tp.npz
2009-01-04_mfx.npz
2009-01-04_tp.npz
2009-01-05_mfx.npz
2009-01-05_tp.npz
2009-01-06_mfx.npz
2009-01-06_tp.npz
2009-01-07_mfx.npz
2009-01-08_mfx.npz
2009-01-09_mfx.npz
2009-01-10_mfx.npz
2009-01-11_mfx.npz
2009-01-12_mfx.npz
2009-01-13_mfx.npz
2009-01-14_mfx.npz
2009-01-15_mfx.npz
2009-01-16_mfx.npz
2009-01-17_mfx.npz
2009-01-18_mfx.npz
2009-01-19_mfx.npz
2009-01-20_mfx.npz
2009-01-21_mfx.npz
2009-01-22_mfx.npz
2009-01-23_mfx.npz
2009-01-24_mfx.npz
2009-01-25_mfx.npz
2009-01-26_mfx.npz
2009-01-27_mfx.npz
2009-01-28_mfx.npz
2009-01-29_mfx.npz
2009-01-30_mfx.npz
2009-01-31_mfx.npz
concat data
save to net_cdf
All data has been combined into a single .nc file with coordinates.


In [27]:
lon = "coordinates_40km_grid/Airs_nh_lon_40_grid.csv"
lat = "coordinates_40km_grid/Airs_nh_lat_40_grid.csv"

# read lon file into numpy array
longitudes = np.loadtxt(lon, delimiter=',', unpack=True)
latitudes = np.loadtxt(lon, delimiter=',', unpack=True)

# Assuming your original arrays are called "latitude", "longitude", and "mfx"
# and have shapes (501, 501)

# Reshape the latitude and longitude arrays to have shapes (501, 1) and (1, 501), respectively
latitude_reshaped = np.reshape(latitudes, (501, 1))
longitude_reshaped = np.reshape(longitudes, (1, 501))

# Broadcast the reshaped arrays to have shapes (501, 501)
latitude_broadcasted = np.broadcast_to(latitude_reshaped, (501, 501))
longitude_broadcasted = np.broadcast_to(longitude_reshaped, (501, 501))

# # Reshape the mfx array to have shape (501, 501, 1)
# mfx_reshaped = np.reshape(mfx, (501, 501, 1))

# # Concatenate the reshaped arrays along the third axis to create the final array with shape (501, 501, 3)
# final_array = np.concatenate((latitude_broadcasted, longitude_broadcasted, mfx_reshaped), axis=2)

ValueError: cannot reshape array of size 251001 into shape (501,1)